# PDAN Model Analysis and Visualization

This notebook provides interactive analysis of the trained PDAN model results.

In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import torch
import torch.nn.functional as F
from train_pdan_lightning import PDANLightningModule
from evaluate_pdan import PDANEvaluator

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

%matplotlib inline

ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

## 1. Load Model and Data

In [ ]:
# Configuration
checkpoint_path = "./lightning_logs/checkpoints/pdan-epoch=39-val_map=10.39.ckpt"
data_root = "/data/1_personal/4_SWWOO/actiondetect/PDAN/data"
split_file = "/data/1_personal/4_SWWOO/actiondetect/PDAN/data/charades.json"

# Load evaluator
evaluator = PDANEvaluator(
    checkpoint_path=checkpoint_path,
    data_root=data_root,
    split_file=split_file,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print(f"Model loaded successfully!")
print(f"Dataset: {len(evaluator.dataset)} samples")

## 2. Run Evaluation

In [ ]:
# Run evaluation
results = evaluator.evaluate_model()

print(f"\nEvaluation Results:")
print(f"Mean Average Precision (mAP): {results['mean_ap']:.2f}%")
print(f"Number of videos: {len(results['video_ids'])}")
print(f"Number of action classes: {len(results['ap_scores'])}")

## 3. Detailed Analysis

In [ ]:
# AP scores analysis
ap_scores = results['ap_scores'] * 100

print("AP Score Statistics:")
print(f"Mean: {np.mean(ap_scores):.2f}%")
print(f"Std: {np.std(ap_scores):.2f}%")
print(f"Min: {np.min(ap_scores):.2f}%")
print(f"Max: {np.max(ap_scores):.2f}%")
print(f"Median: {np.median(ap_scores):.2f}%")

# Show top and bottom performing classes
top_classes = np.argsort(ap_scores)[-10:]
bottom_classes = np.argsort(ap_scores)[:10]

print("\nTop 10 performing classes:")
for i, class_idx in enumerate(reversed(top_classes)):
    print(f"{i+1:2d}. Class {class_idx:3d}: {ap_scores[class_idx]:5.2f}%")

print("\nBottom 10 performing classes:")
for i, class_idx in enumerate(bottom_classes):
    print(f"{i+1:2d}. Class {class_idx:3d}: {ap_scores[class_idx]:5.2f}%")

## 4. Interactive Visualizations

In [ ]:
# Plot AP distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram
ax1.hist(ap_scores, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(x=np.mean(ap_scores), color='red', linestyle='--', 
           label=f'Mean: {np.mean(ap_scores):.1f}%')
ax1.set_title('Distribution of AP Scores')
ax1.set_xlabel('Average Precision (%)')
ax1.set_ylabel('Number of Classes')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot
ax2.boxplot(ap_scores, vert=True)
ax2.set_title('AP Score Distribution (Box Plot)')
ax2.set_ylabel('Average Precision (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Per-video analysis
video_data = []
for video_id, result in results['results_per_video'].items():
    preds = result['predictions']
    labels = result['labels']
    
    video_data.append({
        'video_id': video_id,
        'duration': result['duration'],
        'fps': result['fps'],
        'num_frames': preds.shape[0],
        'mean_prediction': np.mean(preds),
        'max_prediction': np.max(preds),
        'num_active_classes': np.sum(np.max(preds, axis=0) > 0.1),
        'num_gt_actions': np.sum(labels > 0.5)
    })

video_df = pd.DataFrame(video_data)

# Video statistics
print("Video Dataset Statistics:")
print(video_df.describe())

In [ ]:
# Correlation analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Duration vs Mean Prediction
axes[0,0].scatter(video_df['duration'], video_df['mean_prediction'], alpha=0.6)
axes[0,0].set_xlabel('Video Duration (seconds)')
axes[0,0].set_ylabel('Mean Prediction Score')
axes[0,0].set_title('Duration vs Mean Prediction')
axes[0,0].grid(True, alpha=0.3)

# Number of frames vs Max prediction
axes[0,1].scatter(video_df['num_frames'], video_df['max_prediction'], alpha=0.6, color='orange')
axes[0,1].set_xlabel('Number of Frames')
axes[0,1].set_ylabel('Max Prediction Score')
axes[0,1].set_title('Frames vs Max Prediction')
axes[0,1].grid(True, alpha=0.3)

# GT actions vs Active classes
axes[1,0].scatter(video_df['num_gt_actions'], video_df['num_active_classes'], alpha=0.6, color='green')
axes[1,0].set_xlabel('Number of GT Actions')
axes[1,0].set_ylabel('Number of Active Classes (pred > 0.1)')
axes[1,0].set_title('Ground Truth vs Predicted Active Classes')
axes[1,0].grid(True, alpha=0.3)

# FPS distribution
axes[1,1].hist(video_df['fps'], bins=15, alpha=0.7, color='purple', edgecolor='black')
axes[1,1].set_xlabel('FPS (Features Per Second)')
axes[1,1].set_ylabel('Number of Videos')
axes[1,1].set_title('Feature Extraction FPS Distribution')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Temporal Analysis for Sample Videos

In [ ]:
# Select an interesting video for detailed analysis
sample_video_id = list(results['results_per_video'].keys())[0]
sample_result = results['results_per_video'][sample_video_id]

predictions = sample_result['predictions']  # [T, 157]
labels = sample_result['labels']  # [T, 157]
duration = sample_result['duration']

print(f"Analyzing video: {sample_video_id}")
print(f"Duration: {duration:.2f} seconds")
print(f"Number of frames: {predictions.shape[0]}")
print(f"FPS: {sample_result['fps']:.2f}")

# Find most active classes
class_activity = np.sum(predictions, axis=0)
top_active_classes = np.argsort(class_activity)[-5:]

# Create time axis
time_axis = np.linspace(0, duration, predictions.shape[0])

# Plot temporal predictions
plt.figure(figsize=(15, 8))

for i, class_idx in enumerate(reversed(top_active_classes)):
    plt.plot(time_axis, predictions[:, class_idx], 
            label=f'Class {class_idx}', linewidth=2, alpha=0.8)

plt.title(f'Temporal Action Predictions - Video: {sample_video_id}')
plt.xlabel('Time (seconds)')
plt.ylabel('Prediction Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Show ground truth if available
gt_actions = np.where(labels > 0.5)
if len(gt_actions[0]) > 0:
    plt.figure(figsize=(15, 6))
    plt.scatter([time_axis[frame] for frame in gt_actions[0]], 
               gt_actions[1], 
               c='red', s=30, alpha=0.6, label='Ground Truth')
    plt.title(f'Ground Truth Actions - Video: {sample_video_id}')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Action Class')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No ground truth actions found for this video.")

## 6. Model Performance Summary

In [ ]:
# Create performance summary
summary = {
    'Model Performance': {
        'Mean Average Precision (mAP)': f"{results['mean_ap']:.2f}%",
        'Number of Classes': len(results['ap_scores']),
        'Number of Videos': len(results['video_ids']),
        'Best Class AP': f"{np.max(ap_scores):.2f}%",
        'Worst Class AP': f"{np.min(ap_scores):.2f}%",
        'AP Standard Deviation': f"{np.std(ap_scores):.2f}%"
    },
    'Dataset Statistics': {
        'Average Video Duration': f"{video_df['duration'].mean():.2f} seconds",
        'Average Number of Frames': f"{video_df['num_frames'].mean():.1f}",
        'Average FPS': f"{video_df['fps'].mean():.2f}",
        'Total Frames Processed': int(video_df['num_frames'].sum())
    }
}

# Display summary
print("=" * 50)
print("PDAN MODEL EVALUATION SUMMARY")
print("=" * 50)

for category, metrics in summary.items():
    print(f"\n{category}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

print("\n" + "=" * 50)